In [ ]:
"""
Calculate NDVI and Green Space Statistics for Central London

This script calculates the Normalized Difference Vegetation Index (NDVI) from 
processed Sentinel-2 imagery and performs spatial analysis to quantify green space 
coverage across central London boroughs. It generates visualisations and statistics
on vegetation distribution.

Data sources:
- Processed Sentinel-2 bands from clip_sentinel_data.py
- Borough boundaries from process_london_boundaries.py

"""

import os
import numpy as np
import rasterio
from rasterio.mask import mask
from rasterio.features import shapes
from shapely.geometry import shape, mapping
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from skimage import measure

# Define Satellite Image Folder - Processed Data
sentinel_processed_folder = "../Data/Processed/Sentinel_2/07_April_25"

# Extract date part from the path to use in output paths
date_part = os.path.basename(sentinel_processed_folder)

# Create date-specific output folders
ndvi_output_folder = f"../Data/Processed/NDVI/{date_part}"
ndvi_results_folder = f"../Results/NDVI/{date_part}"

# Use the individual borough boundaries in UTM projection
borough_path = "../Data/Processed/London_Borough/central_london_boroughs_utm.geojson"

# Create output directories
os.makedirs(ndvi_output_folder, exist_ok=True)
os.makedirs(ndvi_results_folder, exist_ok=True)

# Auto-detect and load the clipped NIR and Red bands
all_processed_files = os.listdir(sentinel_processed_folder)
red_band_files = [f for f in all_processed_files if "B04_10m_clipped.tif" in f]
nir_band_files = [f for f in all_processed_files if "B08_10m_clipped.tif" in f]

if not red_band_files or not nir_band_files:
    raise FileNotFoundError(f"Required band files not found in {sentinel_processed_folder}. Please run the clipping script first.")

# Use the first matching file if multiple are found
red_path = os.path.join(sentinel_processed_folder, red_band_files[0])
nir_path = os.path.join(sentinel_processed_folder, nir_band_files[0])

print(f"Using RED band: {os.path.basename(red_path)}")
print(f"Using NIR band: {os.path.basename(nir_path)}")

# Load Red and NIR bands
with rasterio.open(red_path) as src_red:
    red_band = src_red.read(1).astype(float)
    red_meta = src_red.meta.copy()

with rasterio.open(nir_path) as src_nir:
    nir_band = src_nir.read(1).astype(float)

# Calculate NDVI

# NDVI = (NIR - Red) / (NIR + Red)
denominator = nir_band + red_band
valid_mask = denominator > 0.001
ndvi = np.zeros_like(red_band)
ndvi[valid_mask] = (nir_band[valid_mask] - red_band[valid_mask]) / denominator[valid_mask]
ndvi = np.clip(ndvi, -1.0, 1.0)

# Save NDVI as GeoTIFF
ndvi_path = os.path.join(ndvi_output_folder, f"central_london_ndvi_{date_part}.tif")
ndvi_meta = red_meta.copy()
ndvi_meta.update({'dtype': 'float32', 'count': 1, 'nodata': -9999})

with rasterio.open(ndvi_path, 'w', **ndvi_meta) as dst:
    dst.write(ndvi.astype(np.float32), 1)
print(f"\nNDVI saved to: {ndvi_path}")

# Create NDVI visualisation
colors = [(0.8, 0.2, 0.2), (0.9, 0.9, 0.4), (0.4, 0.8, 0.2), (0.0, 0.4, 0.0)]
ndvi_cmap = LinearSegmentedColormap.from_list('ndvi_cmap', colors, N=256)

plt.figure(figsize=(10, 10))
plt.imshow(ndvi, cmap=ndvi_cmap, vmin=-0.2, vmax=0.8)
plt.colorbar(label='NDVI')
plt.title('NDVI - Central London', fontsize=14)
plt.axis('off')
plt.savefig(os.path.join(ndvi_results_folder, f"ndvi_map_{date_part}.png"), dpi=300, bbox_inches='tight')
plt.close()

# Create NDVI histogram
plt.figure(figsize=(10, 6))
plt.hist(ndvi.flatten(), bins=100, range=(-0.5, 1.0), alpha=0.7, color='green')
plt.axvline(x=0.3, color='red', linestyle='--', label='Vegetation Threshold (0.3)')
plt.xlabel('NDVI Value')
plt.ylabel('Frequency')
plt.title('Distribution of NDVI Values in Central London')
plt.legend()
plt.grid(alpha=0.3)
plt.savefig(os.path.join(ndvi_results_folder, f"ndvi_histogram_{date_part}.png"), dpi=300)
plt.close()

# Create Binary Vegetation Mask
vegetation_threshold = 0.3
vegetation_mask = (ndvi >= vegetation_threshold).astype(np.uint8)

# Save vegetation mask as GeoTIFF
mask_path = os.path.join(ndvi_output_folder, f"vegetation_mask_{date_part}.tif")
mask_meta = ndvi_meta.copy()
mask_meta.update({'dtype': 'uint8', 'nodata': 255})

with rasterio.open(mask_path, 'w', **mask_meta) as dst:
    dst.write(vegetation_mask, 1)
print(f"\nVegetation mask saved to: {mask_path}")

# Calculate overall vegetation coverage
total_pixels = vegetation_mask.size
vegetated_pixels = np.sum(vegetation_mask)
vegetation_percentage = (vegetated_pixels / total_pixels) * 100
print(f"\nTotal area analysed: {total_pixels} pixels")
print(f"\nVegetated area: {vegetated_pixels} pixels ({vegetation_percentage:.2f}%)")

# Visualise vegetation mask
plt.figure(figsize=(10, 10))
plt.imshow(vegetation_mask, cmap='Greens')
plt.title('Vegetation Areas (NDVI >= 0.3)', fontsize=14)
plt.axis('off')
plt.savefig(os.path.join(ndvi_results_folder, f"vegetation_mask_{date_part}.png"), dpi=300, bbox_inches='tight')
plt.close()

# Identify Individual Green Spaces
labeled_array, num_features = measure.label(vegetation_mask, connectivity=2, return_num=True)
print(f"\nFound {num_features} distinct green spaces")

# Remove small green spaces (noise and small patches)
min_size_threshold = 100
component_sizes = np.bincount(labeled_array.ravel())
too_small = component_sizes < min_size_threshold
too_small_mask = too_small[labeled_array]
labeled_array[too_small_mask] = 0

# Relabel after removing small components
labeled_array, num_filtered_features = measure.label(labeled_array > 0, connectivity=2, return_num=True)
print(f"\nAfter filtering small patches: {num_filtered_features} significant green spaces")

# Create visualisation of the different green spaces
plt.figure(figsize=(12, 12))
plt.imshow(labeled_array, cmap='nipy_spectral')
plt.title(f'Distinct Green Spaces (n={num_filtered_features})', fontsize=14)
plt.axis('off')
plt.savefig(os.path.join(ndvi_results_folder, f"green_spaces_labeled_{date_part}.png"), dpi=300, bbox_inches='tight')
plt.close()

# Calculate size of green spaces
unique_labels = np.unique(labeled_array)
unique_labels = unique_labels[unique_labels > 0]  # Skip the background
green_space_sizes = [np.sum(labeled_array == label) for label in unique_labels]

# Get pixel resolution information from the raster metadata
with rasterio.open(mask_path) as src:
    # Get transform for vectoriation
    mask_transform = src.transform
    mask_crs = src.crs
    
    # Calculate pixel area in hectares (10m resolution = 0.01 ha per pixel)
    pixel_width = abs(src.transform[0])
    pixel_height = abs(src.transform[4])
    pixel_area_ha = (pixel_width * pixel_height) / 10000  # Convert m² to ha

# Convert to hectares using accurate pixel dimensions
green_space_hectares = [size * pixel_area_ha for size in green_space_sizes]

# Create DataFrame for green spaces
green_space_df = pd.DataFrame({
    'id': unique_labels, 
    'size_pixels': green_space_sizes,
    'area_hectares': green_space_hectares
})
green_space_df = green_space_df.sort_values('area_hectares', ascending=False).reset_index(drop=True)

# Create a histogram of green space sizes
plt.figure(figsize=(10, 6))
plt.hist(green_space_hectares, bins=30, color='green', alpha=0.7)
plt.xlabel('Green Space Size (hectares)')
plt.ylabel('Frequency')
plt.title('Distribution of Green Space Sizes in Central London')
plt.grid(alpha=0.3)
plt.savefig(os.path.join(ndvi_results_folder, f"green_space_size_distribution_{date_part}.png"), dpi=300)
plt.close()

# Convert vegetation mask to vector polygons for accurate area calculations
with rasterio.open(mask_path) as src:
    # Create polygon features from the vegetation mask
    vegetation_shapes = list(shapes(
        vegetation_mask.astype('uint8'),
        mask=vegetation_mask==1,
        transform=mask_transform
    ))
    
# Convert vegetation shapes to GeoDataFrame
vegetation_polygons = [shape(geom) for geom, val in vegetation_shapes if val == 1]
vegetation_gdf = gpd.GeoDataFrame(
    {'geometry': vegetation_polygons},
    crs=mask_crs
)

# Save vegetation polygons for visualisation/verification
vegetation_vector_path = os.path.join(ndvi_output_folder, f"vegetation_polygons_{date_part}.geojson")
vegetation_gdf.to_file(vegetation_vector_path, driver="GeoJSON")

# Calculate Borough-Level Green Space Statistics using vector intersection

# Check if borough file exists
if not os.path.exists(borough_path):
    raise FileNotFoundError(f"Borough boundaries not found at {borough_path}. Run process_boundaries.py first.")

# Load borough boundaries
boroughs = gpd.read_file(borough_path)

# Ensure the vegetation GeoDataFrame has the same CRS as the boroughs for accurate intersection
if vegetation_gdf.crs != boroughs.crs:
    vegetation_gdf = vegetation_gdf.to_crs(boroughs.crs)

# Convert to a CRS that preserves area for accurate measurements

# For UK, British National Grid (EPSG:27700) is the standard equal-area projection
boroughs_for_area = boroughs.copy().to_crs("EPSG:27700")
vegetation_for_area = vegetation_gdf.copy().to_crs("EPSG:27700")

# Calculate total vegetation area in hectares (for verification)
total_vegetation_area_ha = sum(vegetation_for_area.geometry.area) / 10000
print(f"\nTotal vegetation area (from vector): {total_vegetation_area_ha:.2f} hectares")
print(f"Total vegetation area (from raster): {vegetated_pixels * pixel_area_ha:.2f} hectares")

# Process each borough with vector operations
borough_stats = []
for idx, borough in boroughs_for_area.iterrows():
    borough_name = borough['NAME']
    
    # Skip if geometry is empty or invalid
    if borough.geometry.is_empty:
        print(f"\nWarning: Empty geometry for {borough_name}. Skipping.")
        continue
    
    # Fix invalid geometries
    if not borough.geometry.is_valid:
        print(f"\nFixing invalid geometry for {borough_name}")
        borough.geometry = borough.geometry.buffer(0)
        if not borough.geometry.is_valid:
            print(f"\nCould not fix geometry for {borough_name}. Skipping.")
            continue
    
    # Calculate total borough area in hectares
    total_area_ha = borough.geometry.area / 10000
    
    try:
        # Perform geometric intersection between borough and vegetation
        # This gives us precisely the vegetated areas within this borough
        intersection = gpd.overlay(vegetation_for_area, 
                                  gpd.GeoDataFrame({'geometry': [borough.geometry]}, crs="EPSG:27700"),
                                  how='intersection')
        
        # Calculate green space area from the intersection geometries
        if len(intersection) > 0:
            # Sum the areas of all vegetation polygons within this borough
            green_area_ha = sum(intersection.geometry.area) / 10000
        else:
            green_area_ha = 0
        
        # Calculate percentage
        green_percentage = (green_area_ha / total_area_ha) * 100 if total_area_ha > 0 else 0
        
        # Store results
        borough_stats.append({
            'Borough': borough_name,
            'Total Area (ha)': round(total_area_ha, 2),
            'Green Area (ha)': round(green_area_ha, 2),
            'Green Space (%)': round(green_percentage, 2)
        })
        
    except Exception as e:
        print(f"\nError processing {borough_name}: {e}")
        continue

# Create DataFrame with results
if not borough_stats:
    raise ValueError("No borough statistics could be calculated. Check data alignment.")

borough_stats_df = pd.DataFrame(borough_stats)
borough_stats_df = borough_stats_df.sort_values('Green Space (%)', ascending=False).reset_index(drop=True)


# Save to CSV
borough_stats_csv = os.path.join(ndvi_results_folder, f"borough_green_space_stats_{date_part}.csv")
borough_stats_df.to_csv(borough_stats_csv, index=False)
print(f"\nBorough statistics saved to: {borough_stats_csv}")

# Calculate London-wide statistics
london_total_area = borough_stats_df['Total Area (ha)'].sum()
london_green_area = borough_stats_df['Green Area (ha)'].sum()
london_green_pct = (london_green_area / london_total_area) * 100

# Add London-wide row
london_row = pd.DataFrame([{
    'Borough': 'CENTRAL LONDON TOTAL',
    'Total Area (ha)': round(london_total_area, 2),
    'Green Area (ha)': round(london_green_area, 2),
    'Green Space (%)': round(london_green_pct, 2)
}])

# Add to stats and save updated version
borough_stats_df = pd.concat([borough_stats_df, london_row], ignore_index=True)
borough_stats_df.to_csv(borough_stats_csv, index=False)


# Create a choropleth map of green space percentage by borough

# Convert back to the original CRS for mapping
boroughs_with_stats = boroughs.copy()
boroughs_with_stats = boroughs_with_stats.merge(
    borough_stats_df[borough_stats_df['Borough'] != 'CENTRAL LONDON TOTAL'],
    left_on='NAME',
    right_on='Borough',
    how='left'
)


# NDVI with boundaries.png

fig, ax = plt.subplots(figsize=(10, 10), dpi=150)

# Use a more visually appealing background color
fig.patch.set_facecolor('#f5f5f5')

# Display NDVI 
with rasterio.open(ndvi_path) as src:
    ndvi_data = src.read(1)
    extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]
    
    # Adjust Contrast
    im = ax.imshow(ndvi_data, cmap=ndvi_cmap, extent=extent, vmin=-0.125, vmax=0.67)

# Overlay borough boundaries with opacity
boroughs.boundary.plot(ax=ax, color='black', linewidth=1.5, alpha=0.5)

# Add borough labels with transparency
for idx, row in boroughs.iterrows():
    if not row.geometry.is_empty and row.geometry.is_valid:
        # Get borough centroid for label placement
        centroid = row.geometry.centroid
        
        # Adjust Borough Label
        if row['NAME'] == 'Kensington and Chelsea':
            y_offset1 = -2200  # Adjusted value
            x_offset1 = -1100  # Adjusted value
            label_x = centroid.x + x_offset1
            label_y = centroid.y + y_offset1

        elif row['NAME'] == 'Lambeth':
            y_offset2 = 600  # Adjusted value
            label_x = centroid.x
            label_y = centroid.y + y_offset2

        elif row['NAME'] == 'Wandsworth':
            y_offset3 = 700  # Adjusted value
            label_x = centroid.x
            label_y = centroid.y + y_offset3

        elif row['NAME'] == 'Westminster':
            y_offset4 = 400  # Adjusted value
            label_x = centroid.x
            label_y = centroid.y + y_offset4

        else:
            label_x = centroid.x
            label_y = centroid.y
        
        # Add transparent label background
        ax.text(label_x, label_y, row['NAME'], 
                fontsize=9, ha='center', va='center', 
                bbox=dict(facecolor='white', alpha=0.4, edgecolor='black', 
                          boxstyle='round,pad=0.3', linewidth=0.5))

# Add a colorbar with label
cbar = fig.colorbar(im, ax=ax, pad=0.01, shrink=0.7)
cbar.set_label('NDVI Value', fontsize=10)

# Remove axis completely for clean look
ax.set_axis_off()

# Add title
plt.title('London Green Spaces: Analysis using Satellite Imagery\n', 
        fontsize=13, fontweight='bold', ha='center', va='top', 
        transform=ax.transAxes, color='#333333')

# Create a cleaned date string for display purposes
display_date = date_part.replace("_", " ")

# Add data source credits
ax.text(0.02, 0.01, f'Data Source: Sentinel-2 Satellite Imagery ({display_date})', 
        fontsize=8, ha='left', va='bottom', 
        transform=ax.transAxes, color='#333333')

# Add a brief explanation of NDVI
ax.text(0.98, 0.01, 'NDVI: Higher values indicate\ndenser greenery', 
        fontsize=8, ha='right', va='bottom', 
        transform=ax.transAxes, color='#333333',
        )

# Use tight layout to maximize map area
plt.tight_layout()

# Save with white border padding
plt.savefig(os.path.join(ndvi_results_folder, f"ndvi_with_boundaries_{date_part}.png"), 
           dpi=300, bbox_inches='tight', pad_inches=0.2, 
           facecolor=fig.get_facecolor())


# Bar Chart
fig, ax = plt.subplots(figsize=(12, 10))
boroughs_with_stats.plot(
    column='Green Space (%)',
    cmap='Greens',
    linewidth=0.8,
    ax=ax,
    edgecolor='black',
    legend=True,
    missing_kwds={'color': 'lightgray'},
    legend_kwds={'label': 'Green Space (%)', 'orientation': 'horizontal'}
)
# Add borough labels
for idx, row in boroughs_with_stats.iterrows():
    try:
        # Only add label if the borough has a valid centroid
        if not row.geometry.is_empty and row.geometry.is_valid:
            centroid = row.geometry.centroid
            ax.text(centroid.x, centroid.y, row['NAME'], 
                    fontsize=8, ha='center', va='center', 
                    bbox=dict(facecolor='white', alpha=0.5, edgecolor='none', boxstyle='round,pad=0.3'))
    except Exception as e:
        print(f"\nCould not add label for {row['NAME']}: {e}")

plt.title('Green Space Percentage by Borough', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(ndvi_results_folder, f"borough_green_space_map_{date_part}.png"), dpi=300, bbox_inches='tight')
plt.close()

# Bar chart of green space percentage by borough
plt.figure(figsize=(12, 8))
data = borough_stats_df[borough_stats_df['Borough'] != 'CENTRAL LONDON TOTAL'].copy()
colors = ['green' if x >= london_green_pct else 'lightgreen' for x in data['Green Space (%)']]

plt.bar(data['Borough'], data['Green Space (%)'], color=colors)
plt.axhline(y=london_green_pct, color='red', linestyle='--', 
            label=f'Central London Average ({london_green_pct:.1f}%)')
plt.xlabel('Borough')
plt.ylabel('Green Space (%)')
plt.title('Green Space % by Borough')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.legend()
plt.savefig(os.path.join(ndvi_results_folder, f"bar_chart_borough_green_space_{date_part}.png"), dpi=300)

#Analysis Complete
print(f"\nAnalysis and Visuals complete for {date_part}.")